In [20]:
import sys
import time
from datetime import datetime, date
from pathlib import Path
import math

import pandas as pd
import mlflow
from domino.data_sources import DataSourceClient

mlflow.set_experiment("Populate DB (bond_inventory)")
ds = DataSourceClient().get_datasource("market_data")

BATCH_SIZE = 1000

# ─────────────────────────────────────────────────────────────────────────────
def fetch_active_bonds(inventory_date: date) -> list[dict]:
    sql = f"""
    SELECT *
      FROM treasury_auction_results
     WHERE security_type IN ('Note', 'Bill', 'Bond')
       AND issue_date < '{inventory_date}'
       AND maturity_date > '{inventory_date}'
    """
    return ds.query(sql).to_pandas().to_dict(orient="records")

def sanitize_quantity(val):
    try:
        q = float(val)
        if math.isnan(q) or math.isinf(q):
            return 0.0
        return round(q * 0.01, 2)
    except:
        return 0.0

def quote(val):
    if val is None:
        return "NULL"
    if isinstance(val, float) and (math.isnan(val) or math.isinf(val)):
        return "NULL"
    if isinstance(val, pd.Timestamp) and pd.isna(val):
        return "NULL"
    if isinstance(val, str):
        v = val.strip().lower()
        if v in {"nan", "null", ""}:
            return "NULL"
        return "'" + val.replace("'", "''") + "'"
    try:
        # Convert strings that can be interpreted as numbers (e.g., '123.45')
        float(val)
        return str(val)
    except:
        return "'" + str(val).replace("'", "''") + "'"

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

def upsert_inventory(records: list[dict], inventory_date: date, batch_size: int = BATCH_SIZE):
    if not records:
        print("⚠️ No active securities for inventory_date:", inventory_date)
        return

    cols = [
        "inventory_date", "cusip", "quantity",
        "security_type", "security_term",
        "issue_date", "maturity_date",
        "int_rate", "int_payment_frequency", "series",
        "price_per100", "auction_date"
    ]

    col_list = ", ".join(cols)
    print(f"🟢 Upserting {len(records)} rows to bond_inventory...")

    for idx, chunk in enumerate(chunks(records, batch_size), start=1):
        print(f"  Batch {idx}: {len(chunk)} rows")
        values_sql = []
        for r in chunk:
            vals = [quote(r.get(c)) for c in cols]
            values_sql.append("(" + ", ".join(vals) + ")")
        values_str = ",\n".join(values_sql)
        set_list = ", ".join([f"{c} = EXCLUDED.{c}" for c in cols if c != "inventory_date" and c != "cusip"])
        sql = f"""
        INSERT INTO bond_inventory ({col_list})
        VALUES
        {values_str}
        ON CONFLICT (inventory_date, cusip)
        DO UPDATE SET
          {set_list};
        """
        ds.query(sql)

def find_mount_root(start: Path, target: str = "mnt") -> Path:
    current = start.resolve()
    while current.name != target:
        if current.parent == current:
            raise FileNotFoundError(f"Could not find folder named '{target}' in parent paths.")
        current = current.parent
    return current

# ─────────────────────────────────────────────────────────────────────────────
def main(inventory_date: str):
    inv_date = datetime.strptime(inventory_date, "%Y-%m-%d").date()

    with mlflow.start_run(run_name=f"Inventory Snapshot {inventory_date}"):
        t0 = time.time()

        auctions = fetch_active_bonds(inv_date)
        print(f"Fetched {len(auctions)} active securities")
        # print(auctions)

        inventory = []
        for r in auctions:
            try:
                q = round(sanitize_quantity(r.get("comp_accepted")) / 1_000_000) * 5_000
            except:
                q = 0.0
            if q > 0:
                inventory.append({
                    "inventory_date": inv_date,
                    "cusip": r.get("cusip"),
                    "quantity": q,
                    "security_type": r.get("security_type"),
                    "security_term": r.get("security_term"),
                    "issue_date": r.get("issue_date"),
                    "maturity_date": r.get("maturity_date"),
                    "int_rate": r.get("int_rate"),
                    "int_payment_frequency": r.get("int_payment_frequency"),
                    "series": r.get("series"),
                    "price_per100": r.get("price_per100"),
                    "auction_date": r.get("auction_date"),
                })
        seen = set()
        deduped_inventory = []
        for row in inventory:
            key = (row["inventory_date"], row["cusip"])
            if key in seen:
                continue
            seen.add(key)
            deduped_inventory.append(row)


        upsert_inventory(deduped_inventory, inv_date)
        duration = time.time() - t0
        mlflow.log_metric("rows_inserted", len(deduped_inventory))
        mlflow.log_metric("duration_seconds", duration)
        print(f"✅ Done in {duration:.1f}s")

        df = pd.DataFrame(deduped_inventory)
        mnt_root = find_mount_root(Path.cwd())
        out = mnt_root / "artifacts" / "results" / f"bond_inventory_{inventory_date}.csv"
        out.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(out, index=False)
        mlflow.log_artifact(str(out), artifact_path="bond_inventory")

# ─────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    main('2025-05-21')


Fetched 597 active securities
🟢 Upserting 355 rows to bond_inventory...
  Batch 1: 355 rows
✅ Done in 1.5s
🏃 View run Inventory Snapshot 2025-05-21 at: http://127.0.0.1:8768/#/experiments/1444/runs/389da65005354f2d8d516fab173089ea
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1444
